In [1]:
%load_ext autoreload
%autoreload 2

import numpy as np
from pydrake.all import (
    DiagramBuilder,
    LinearQuadraticRegulator,
    MeshcatVisualizer,
    MultibodyPlant,
    Parser,
    Propeller,
    PropellerInfo,
    RigidTransform,
    RobotDiagramBuilder,
    SceneGraph,
    Simulator,
    Meshcat,
    StartMeshcat,
    namedview,
)
from pydrake.examples import QuadrotorGeometry, QuadrotorPlant, StabilizingLQRController
import numpy as np

running_as_notebook = True
meshcat = StartMeshcat()

INFO:drake:Meshcat listening for connections at http://localhost:7000


# LQR

### with Euler Angles

In [125]:
from pydrake.all import DiagramBuilder
from pydrake.examples import QuadrotorGeometry as QuadrotorGeometry_EA
from control.UtilLeafSystems import NonConstantVectorSource

from control.QuadrotorControllers import QuadrotorLQR_EA
from sim.Quadrotor import MakeMultibodyQuadrotor_EA
from maths.quaternions import SampleQuaternion
from IPython.display import SVG, display, clear_output, Markdown
import pydot
import time 


Q = np.diag(np.concatenate(([10] * 6, [1] * 6)))
R = np.eye(4)

builder = DiagramBuilder()

# Init Systems
quadrotor, mbp = MakeMultibodyQuadrotor_EA(show_diagram=False)
quadrotor_system = builder.AddSystem(quadrotor)
controller = builder.AddSystem(QuadrotorLQR_EA(quadrotor_system, mbp, Q, R))
goal_state_source = builder.AddSystem(NonConstantVectorSource(12))
ref_action_source = builder.AddSystem(NonConstantVectorSource(4))


# Controller input connections
builder.Connect(
    quadrotor_system.get_output_port(), controller.get_input_port(0)
)
builder.Connect(
    goal_state_source.get_output_port(), controller.get_input_port(1)
)
builder.Connect(
    ref_action_source.get_output_port(), controller.get_input_port(2)
)

# Controller output connections
builder.Connect(
    controller.get_output_port(), quadrotor_system.get_input_port()
)

# Setup visualization
scene_graph = builder.AddSystem(SceneGraph())
quadrotor_geometry_system = QuadrotorGeometry_EA.AddToBuilder(
    builder, quadrotor_system.get_output_port(0), scene_graph
)
meshcat.Delete()
meshcat.ResetRenderMode()
meshcat.SetProperty("/Background", "visible", False)
visualizer = MeshcatVisualizer.AddToBuilder(builder, scene_graph, meshcat)

diagram = builder.Build()
# display(SVG(pydot.graph_from_dot_data(
#             diagram.GetGraphvizString(max_depth=1))[0].create_svg()))

# Set up a simulator to run this diagram
simulator = Simulator(diagram)
simulator.set_target_realtime_rate(1.0)
diagram_context = simulator.get_mutable_context()

# Get the subsystem contexts
quadrotor_context = quadrotor_system.GetMyContextFromRoot(diagram_context)
controller_context = controller.GetMyContextFromRoot(diagram_context)
geometry_context = diagram.GetSubsystemContext(quadrotor_geometry_system, diagram_context)

# Function to print the state vector
def dynamically_update_output(state, control):
    clear_output(wait=True)
    display(Markdown(f"**State:**\n```\n{state}\n```\n**Control:**\n```\n{control}\n```"))

def log_state():
    quadrotor_state_port = quadrotor_system.get_output_port(0)
    control_output_port = controller.get_output_port(0)

    quadrotor_state = quadrotor_state_port.Eval(quadrotor_context)
    control = control_output_port.Eval(controller_context)
    
    dynamically_update_output(quadrotor_state, control)
    
    if np.any(np.isnan(quadrotor_state)):
        raise ValueError("Quadrotor state contains NaN values")

    if np.any(np.isnan(control)):
        raise ValueError("Control input contains NaN values")

def inspect_base_link_state(context, plant):
    inspector = plant.GetMyContextFromRoot(context)
    base_link_frame = plant.GetFrameByName('base_link')
    pose = plant.CalcRelativeTransform(inspector, plant.world_frame(), base_link_frame)
    print(f"Base Link Pose:\n{pose}")

def simulate(diagram_context, initial_state, ref_state, ref_action, duration):
    diagram_context.SetTime(0.0)
    diagram_context.SetContinuousState(initial_state)
    goal_state_source.SetState(ref_state)
    ref_action_source.SetState(ref_action) 
    simulator.Initialize()
    try:
        while diagram_context.get_time() < duration:
            # log_state()

            # Force evaluation of output port to see the state in QuadrotorGeometry
            # quadrotor_geometry_output: FramePoseVector = diagram.GetSubsystemByName('QuadrotorGeometry').get_output_port(0).Eval(geometry_context)
            # quadrotor_geometry_pose = quadrotor_geometry_output.value(quadrotor_geometry_output.ids()[0])

            # position_np = np.array(quadrotor_geometry_pose.translation())
            # rotation_np = np.array(quadrotor_geometry_pose.rotation().matrix())
            # print(f"Pose (Position):\n{position_np}")
            # print(f"Pose (Rotation):\n{rotation_np}")

            # inspect_base_link_state(diagram_context, mbp)

            simulator.AdvanceTo(diagram_context.get_time() + 0.1)
    except ValueError as e:
        print(f"Simulation error: {e}")

# initial_state = np.array([1., 0., 0., 0., 0., 0., 1.5, 0., 0., 0., 0., 0., 0.])
initial_state = np.hstack(
    (np.array([0., 0., 0.8]),
     .5* np.random.randn(3),
    np.array([0., 0., 0., 0., 0., 0.])
    )
)

ref_state = np.array([0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.])
gravity = mbp.gravity_field().gravity_vector()[2]
ref_action = np.array([-controller.mass * gravity / 4 for i in range(4)] )
print(ref_action)
simulate(diagram_context, initial_state, ref_state, ref_action, duration = 4.0)

0.775
[1.9006875 1.9006875 1.9006875 1.9006875]


### with Quaternions

In [124]:
from pydrake.all import DiagramBuilder
from control.UtilLeafSystems import NonConstantVectorSource

from control.QuadrotorControllers import QuadrotorLQR
from sim.Quadrotor import MakeMultibodyQuadrotor, QuadrotorGeometry
from maths.quaternions import SampleQuaternion
from IPython.display import SVG, display, clear_output, Markdown
import pydot
import time 

dt = 0.001
Q = np.diag(np.concatenate(([10] * 6, [1] * 6)))
R = np.eye(4)

builder = DiagramBuilder()

# Init Systems
quadrotor, mbp = MakeMultibodyQuadrotor(show_diagram=False)
quadrotor_system = builder.AddSystem(quadrotor)
controller = builder.AddSystem(QuadrotorLQR(quadrotor_system, mbp, Q, R))
goal_state_source = builder.AddSystem(NonConstantVectorSource(13))
ref_action_source = builder.AddSystem(NonConstantVectorSource(4))

# Controller input connections
builder.Connect(
    quadrotor_system.get_output_port(), controller.get_input_port(0)
)
builder.Connect(
    goal_state_source.get_output_port(), controller.get_input_port(1)
)
builder.Connect(
    ref_action_source.get_output_port(), controller.get_input_port(2)
)

# Controller output connections
builder.Connect(
    controller.get_output_port(), quadrotor_system.get_input_port()
)

# Setup visualization
scene_graph = builder.AddSystem(SceneGraph())
quadrotor_geometry_system = QuadrotorGeometry.AddToBuilder(
    builder, quadrotor_system.get_output_port(0), scene_graph, dt
)
meshcat.Delete()
meshcat.ResetRenderMode()
meshcat.SetProperty("/Background", "visible", False)
visualizer = MeshcatVisualizer.AddToBuilder(builder, scene_graph, meshcat)

diagram = builder.Build()
# display(SVG(pydot.graph_from_dot_data(
#             diagram.GetGraphvizString(max_depth=1))[0].create_svg()))

# Set up a simulator to run this diagram
simulator = Simulator(diagram)
simulator.set_target_realtime_rate(1.0)
diagram_context = simulator.get_mutable_context()

# Get the subsystem contexts
quadrotor_context = quadrotor_system.GetMyContextFromRoot(diagram_context)
controller_context = controller.GetMyContextFromRoot(diagram_context)
geometry_context = diagram.GetSubsystemContext(quadrotor_geometry_system, diagram_context)

# Function to print the state vector
def dynamically_update_output(state, control):
    clear_output(wait=True)
    display(Markdown(f"**State:**\n```\n{state}\n```\n**Control:**\n```\n{control}\n```"))

def log_state():
    quadrotor_state_port = quadrotor_system.get_output_port(0)
    control_output_port = controller.get_output_port(0)

    quadrotor_state = quadrotor_state_port.Eval(quadrotor_context)
    control = control_output_port.Eval(controller_context)
    
    dynamically_update_output(quadrotor_state, control)
    
    if np.any(np.isnan(quadrotor_state)):
        raise ValueError("Quadrotor state contains NaN values")

    if np.any(np.isnan(control)):
        raise ValueError("Control input contains NaN values")

def inspect_base_link_state(context, plant):
    inspector = plant.GetMyContextFromRoot(context)
    base_link_frame = plant.GetFrameByName('base_link')
    pose = plant.CalcRelativeTransform(inspector, plant.world_frame(), base_link_frame)
    print(f"Base Link Pose:\n{pose}")

def simulate(diagram_context, initial_state, ref_state, ref_action, duration):
    diagram_context.SetTime(0.0)
    diagram_context.SetContinuousState(initial_state)
    goal_state_source.SetState(ref_state)
    ref_action_source.SetState(ref_action) 
    simulator.Initialize()
    try:
        while diagram_context.get_time() < duration:
            # log_state()

            # Force evaluation of output port to see the state in QuadrotorGeometry
            # quadrotor_geometry_output: FramePoseVector = diagram.GetSubsystemByName('QuadrotorGeometry').get_output_port(0).Eval(geometry_context)
            # quadrotor_geometry_pose = quadrotor_geometry_output.value(quadrotor_geometry_output.ids()[0])

            # position_np = np.array(quadrotor_geometry_pose.translation())
            # rotation_np = np.array(quadrotor_geometry_pose.rotation().matrix())
            # print(f"Pose (Position):\n{position_np}")
            # print(f"Pose (Rotation):\n{rotation_np}")

            # inspect_base_link_state(diagram_context, mbp)

            simulator.AdvanceTo(diagram_context.get_time() + 0.1)
    except ValueError as e:
        print(f"Simulation error: {e}")

# initial_state = np.array([1., 0., 0., 0., 0., 0., 1.5, 0., 0., 0., 0., 0., 0.])
initial_state = np.hstack(
    (SampleQuaternion(near_identity=True),
    np.array([1., 0., 0., 0., 0., 0., 0., 0., 0.])
    )
)

ref_state = np.array([1., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.])
gravity = mbp.gravity_field().gravity_vector()[2]
ref_action = np.array([-controller.mass * gravity / 4 for i in range(4)] )
print(ref_action)
simulate(diagram_context, initial_state, ref_state, ref_action, duration = 4.0)

TypeError: QuadrotorGeometry.AddToBuilder() takes 4 positional arguments but 5 were given

# iLQR

### With Euler Angles

In [14]:
from pydrake.all import DiagramBuilder
from pydrake.examples import QuadrotorGeometry as QuadrotorGeometry_EA

from control.QC import QuadrotorController_EA
from control.UtilLeafSystems import NonConstantVectorSource

from sim.Quadrotor import MakeMultibodyQuadrotor_EA
from maths.quaternions import SampleQuaternion
from IPython.display import SVG, display, clear_output, Markdown
np.set_printoptions(linewidth=500)

Q = np.diag(np.concatenate(([10.] * 6, [1.] * 6)))
R = np.diag([1.0] * 4)
dt = 0.01

builder = DiagramBuilder()

# Init Systemds
quadrotor, mbp = MakeMultibodyQuadrotor_EA(show_diagram = False)
quadrotor_system = builder.AddSystem(quadrotor)
iLQRparams = {
    "quadrotor" : quadrotor_system,
    "multibody_plant" : mbp,
    "Q" : Q,
    "R" : R,
    "Qf" : Q,
    "N" : 10,
    "dt" : dt,
    "max_iter" : 6,
    "regu_init" : 100,
    "min_regu": 0.001,
    "max_regu": 10000,
    "max_linesearch_iters": 5,
    "analytic_model": False
}


# print(mbp.GetStateNames())

controller = builder.AddSystem(QuadrotorController_EA(**iLQRparams))
goal_state_source = builder.AddSystem(NonConstantVectorSource(12))


# Controller input connections
builder.Connect(
    quadrotor_system.get_output_port(), controller.get_input_port(0)
)
builder.Connect(
    goal_state_source.get_output_port(), controller.get_input_port(1)
)


# Controller output connections
builder.Connect(
    controller.get_output_port(), quadrotor_system.get_input_port()
)

# Setup visualization
scene_graph = builder.AddSystem(SceneGraph())
quadrotor_geometry_system = QuadrotorGeometry_EA.AddToBuilder(
    builder, quadrotor_system.get_output_port(0), scene_graph
)
meshcat.Delete()
meshcat.ResetRenderMode()
meshcat.SetProperty("/Background", "visible", False)
visualizer = MeshcatVisualizer.AddToBuilder(builder, scene_graph, meshcat)

diagram = builder.Build()
# display(SVG(pydot.graph_from_dot_data(
#             diagram.GetGraphvizString(max_depth=1))[0].create_svg()))

# Set up a simulator to run this diagram
simulator = Simulator(diagram)
# simulator.set_target_realtime_rate(1.0)
diagram_context = simulator.get_mutable_context()

# Get the subsystem contexts
quadrotor_context = quadrotor_system.GetMyContextFromRoot(diagram_context)
controller_context = controller.GetMyContextFromRoot(diagram_context)
geometry_context = diagram.GetSubsystemContext(quadrotor_geometry_system, diagram_context)

# Function to print the state vector
def dynamically_update_output(state, control, time):
    clear_output(wait=True)
    display(Markdown(f"**Time:**\n```\n{time}\n```**State:**\n```\n{state}\n```\n**Control:**\n```\n{control}\n```"))

def log_state(time):
    quadrotor_state_port = quadrotor_system.get_output_port(0)
    control_output_port = controller.get_output_port(0)

    quadrotor_state = quadrotor_state_port.Eval(quadrotor_context)
    control = control_output_port.Eval(controller_context)
    
    dynamically_update_output(quadrotor_state, control, time)
    
    if np.any(np.isnan(quadrotor_state)):
        raise ValueError("Quadrotor state contains NaN values")

    if np.any(np.isnan(control)):
        raise ValueError("Control input contains NaN values")

def inspect_base_link_state(context, plant):
    inspector = plant.GetMyContextFromRoot(context)
    base_link_frame = plant.GetFrameByName('base_link')
    pose = plant.CalcRelativeTransform(inspector, plant.world_frame(), base_link_frame)
    print(f"Base Link Pose:\n{pose}")

def simulate(diagram_context, initial_state, ref_state, duration):
    diagram_context.SetTime(0.0)
    diagram_context.SetContinuousState(initial_state)
    goal_state_source.SetState(ref_state)
    simulator.Initialize()
    try:
        while diagram_context.get_time() < duration:
            # log_state(diagram_context.get_time())
            sim_time = diagram_context.get_time() + iLQRparams["dt"]
            simulator.AdvanceTo(sim_time)
            print(f"Time: {sim_time}")
    except ValueError as e:
        print(f"Simulation error: {e}")

# initial_state = np.array([1., 0., 0., 0., 0., 0., 1.5, 0., 0., 0., 0., 0., 0.])
initial_state = np.hstack(
    (np.array([0., 0., 0.8]),
     .5* np.random.randn(3),
    np.array([0., 0., 0., 0., 0., 0.])
    )
)
ref_state = np.array([0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

print("Begin simulation")
simulate(diagram_context, initial_state, ref_state, duration = 4.0)

Begin simulation
Time: 0.01
Time: 0.02
Time: 0.03
Time: 0.04
Time: 0.05
Time: 0.060000000000000005
Time: 0.07
Time: 0.08
Time: 0.09
Time: 0.09999999999999999
Time: 0.10999999999999999
Time: 0.11999999999999998
Time: 0.12999999999999998
Time: 0.13999999999999999
Time: 0.15
Time: 0.16
Time: 0.17
Time: 0.18000000000000002
Time: 0.19000000000000003
Time: 0.20000000000000004
Time: 0.21000000000000005
Time: 0.22000000000000006
Time: 0.23000000000000007
Time: 0.24000000000000007
Time: 0.25000000000000006
Time: 0.26000000000000006
Time: 0.2700000000000001
Time: 0.2800000000000001
Time: 0.2900000000000001
Time: 0.3000000000000001
Time: 0.3100000000000001
Time: 0.3200000000000001
Time: 0.3300000000000001
Time: 0.34000000000000014
Time: 0.35000000000000014
Time: 0.36000000000000015
Time: 0.37000000000000016
Time: 0.38000000000000017
Time: 0.3900000000000002
Time: 0.4000000000000002
Time: 0.4100000000000002
Time: 0.4200000000000002
Time: 0.4300000000000002
Time: 0.4400000000000002
Time: 0.45000000

KeyboardInterrupt: 

### With Quaternions

In [ ]:
from pydrake.all import DiagramBuilder, FramePoseVector

from control.QuadrotorControllers import QuadrotoriLQR
from control.UtilLeafSystems import NonConstantVectorSource

from sim.Quadrotor import MakeMultibodyQuadrotor, QuadrotorGeometry
from maths.quaternions import SampleQuaternion
from IPython.display import SVG, display, clear_output, Markdown
np.set_printoptions(linewidth=500)

Q = np.diag(np.concatenate(([10.] * 6, [1.] * 6)))
R = np.diag([0.1] * 4)

builder = DiagramBuilder()

# Init Systems
quadrotor, mbp = MakeMultibodyQuadrotor(show_diagram=False)
quadrotor_system = builder.AddSystem(quadrotor)
# print(mbp.GetStateNames())

iLQRparams = {
    "quadrotor" : quadrotor_system,
    "multibody_plant" : mbp,
    "Q" : Q,
    "R" : R,
    "Qf" : Q,
    "Tf" : 0.5,
    "dt" : 0.1,
    "max_iters" : 5,
    "init_regu" : 0.1,
}

controller = builder.AddSystem(QuadrotoriLQR(**iLQRparams))
goal_state_source = builder.AddSystem(NonConstantVectorSource(13))


# Controller input connections
builder.Connect(
    quadrotor_system.get_output_port(), controller.get_input_port(0)
)
builder.Connect(
    goal_state_source.get_output_port(), controller.get_input_port(1)
)


# Controller output connections
builder.Connect(
    controller.get_output_port(), quadrotor_system.get_input_port()
)

# Setup visualization
scene_graph = builder.AddSystem(SceneGraph())
quadrotor_geometry_system = QuadrotorGeometry.AddToBuilder(
    builder, quadrotor_system.get_output_port(0), scene_graph
)
meshcat.Delete()
meshcat.ResetRenderMode()
meshcat.SetProperty("/Background", "visible", False)
visualizer = MeshcatVisualizer.AddToBuilder(builder, scene_graph, meshcat)

diagram = builder.Build()
# display(SVG(pydot.graph_from_dot_data(
#             diagram.GetGraphvizString(max_depth=1))[0].create_svg()))

# Set up a simulator to run this diagram
simulator = Simulator(diagram)
simulator.set_target_realtime_rate(1.0)
diagram_context = simulator.get_mutable_context()

# Get the subsystem contexts
quadrotor_context = quadrotor_system.GetMyContextFromRoot(diagram_context)
controller_context = controller.GetMyContextFromRoot(diagram_context)
geometry_context = diagram.GetSubsystemContext(quadrotor_geometry_system, diagram_context)

# Function to print the state vector
def dynamically_update_output(state, control, time):
    clear_output(wait=True)
    display(Markdown(f"**Time:**\n```\n{time}\n```**State:**\n```\n{state}\n```\n**Control:**\n```\n{control}\n```"))

def log_state(time):
    quadrotor_state_port = quadrotor_system.get_output_port(0)
    control_output_port = controller.get_output_port(0)

    quadrotor_state = quadrotor_state_port.Eval(quadrotor_context)
    control = control_output_port.Eval(controller_context)
    
    dynamically_update_output(quadrotor_state, control, time)
    
    if np.any(np.isnan(quadrotor_state)):
        raise ValueError("Quadrotor state contains NaN values")

    if np.any(np.isnan(control)):
        raise ValueError("Control input contains NaN values")

def inspect_base_link_state(context, plant):
    inspector = plant.GetMyContextFromRoot(context)
    base_link_frame = plant.GetFrameByName('base_link')
    pose = plant.CalcRelativeTransform(inspector, plant.world_frame(), base_link_frame)
    print(f"Base Link Pose:\n{pose}")

def simulate(diagram_context, initial_state, ref_state, duration):
    diagram_context.SetTime(0.0)
    diagram_context.SetContinuousState(initial_state)
    goal_state_source.SetState(ref_state)
    simulator.Initialize()
    try:
        while diagram_context.get_time() < duration:
            # log_state(diagram_context.get_time())
            simulator.AdvanceTo(diagram_context.get_time() + iLQRparams["dt"])
    except ValueError as e:
        print(f"Simulation error: {e}")

# initial_state = np.array([1., 0., 0., 0., 0., 0., 1.5, 0., 0., 0., 0., 0., 0.])
initial_state = np.hstack(
    (SampleQuaternion(near_identity=True),
    np.array([1., 0., 0., 0., 0., 0., 0., 0., 0.])
    )
)
ref_state = np.array([1., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.])
simulate(diagram_context, initial_state, ref_state, duration = 4.0)

In [85]:

from scipy.spatial.transform import Rotation

euler_angles = np.random.uniform(0, 2*np.pi, 3)

r, p, y = euler_angles
cr, cp, cy = np.cos(euler_angles)
sr, sp, sy = np.sin(euler_angles)

# Rotation matrix from body frame to world frame
R_NB = np.array([[      cp*cy,        cy*sp*sr - sy*cr,    cy*sp*cr + sy*sr],
                [       sy*cp,        sy*sp*sr + cy*cr,    sy*sp*cr - cy*sr],
                [        -sp,               cp*sr,               cr*cp    ]])

GT =  Rotation.from_euler('xyz', euler_angles, degrees=False).as_matrix()
print(R_NB-GT)

[[ 1.94289029e-16 -3.33066907e-16  1.11022302e-16]
 [ 9.28077060e-17 -1.66533454e-16 -1.11022302e-16]
 [ 1.11022302e-16  8.32667268e-17  1.24900090e-16]]


In [2]:
from pydrake.all import DiagramBuilder
from pydrake.examples import QuadrotorGeometry as QuadrotorGeometry_EA
from control.UtilLeafSystems import NonConstantVectorSource

from control.QuadrotorControllers import QuadrotorLQR_EA
from sim.Quadrotor import MakeMultibodyQuadrotor_EA
from maths.quaternions import SampleQuaternion
from IPython.display import SVG, display, clear_output, Markdown
from time import time 


def timeit(func):
    def wrapper(*args, **kwargs):
        start_time = time()  # Start time before the function call
        result = func(*args, **kwargs)  # Call the function
        end_time = time()  # End time after the function call
        elapsed_time = end_time - start_time  # Calculate the time difference
        print(f"Function '{func.__name__}' executed in {elapsed_time:.4f} seconds")
        return result  # Return the original function's result
    return wrapper

Q = np.diag(np.concatenate(([10] * 6, [1] * 6)))
R = np.eye(4)

builder = DiagramBuilder()

# Init Systems
quadrotor, mbp = MakeMultibodyQuadrotor_EA(show_diagram=False)
quadrotor_system = builder.AddSystem(quadrotor)
controller = builder.AddSystem(QuadrotorLQR_EA(quadrotor_system, mbp, Q, R))
goal_state_source = builder.AddSystem(NonConstantVectorSource(12))
ref_action_source = builder.AddSystem(NonConstantVectorSource(4))


# Controller input connections
builder.Connect(
    quadrotor_system.get_output_port(), controller.get_input_port(0)
)
builder.Connect(
    goal_state_source.get_output_port(), controller.get_input_port(1)
)
builder.Connect(
    ref_action_source.get_output_port(), controller.get_input_port(2)
)

# Controller output connections
builder.Connect(
    controller.get_output_port(), quadrotor_system.get_input_port()
)

# Setup visualization
scene_graph = builder.AddSystem(SceneGraph())
quadrotor_geometry_system = QuadrotorGeometry_EA.AddToBuilder(
    builder, quadrotor_system.get_output_port(0), scene_graph
)
meshcat.Delete()
meshcat.ResetRenderMode()
meshcat.SetProperty("/Background", "visible", False)
visualizer = MeshcatVisualizer.AddToBuilder(builder, scene_graph, meshcat)

diagram = builder.Build()
# display(SVG(pydot.graph_from_dot_data(
#             diagram.GetGraphvizString(max_depth=1))[0].create_svg()))

# Set up a simulator to run this diagram
simulator = Simulator(diagram)
simulator.set_target_realtime_rate(1.0)
diagram_context = simulator.get_mutable_context()

# Get the subsystem contexts
quadrotor_context = quadrotor_system.GetMyContextFromRoot(diagram_context)
controller_context = controller.GetMyContextFromRoot(diagram_context)
geometry_context = diagram.GetSubsystemContext(quadrotor_geometry_system, diagram_context)

# Function to print the state vector
def dynamically_update_output(state, control):
    clear_output(wait=True)
    display(Markdown(f"**State:**\n```\n{state}\n```\n**Control:**\n```\n{control}\n```"))

def log_state():
    quadrotor_state_port = quadrotor_system.get_output_port(0)
    control_output_port = controller.get_output_port(0)

    quadrotor_state = quadrotor_state_port.Eval(quadrotor_context)
    control = control_output_port.Eval(controller_context)

    return quadrotor_state, control

def simulate(dt, diagram_context, initial_state, ref_state, ref_action, duration):
    diagram_context.SetTime(0.0)
    diagram_context.SetContinuousState(initial_state)
    goal_state_source.SetState(ref_state)
    ref_action_source.SetState(ref_action) 
    simulator.Initialize()
    try:
        state_log = initial_state
        control_log = None
        
        while diagram_context.get_time() < duration:
            state, control = log_state()
            state_log = np.vstack((state_log, state))
            control_log = np.vstack((control_log, control)) if control_log is not None else control
            simulator.AdvanceTo(diagram_context.get_time() + dt)
    except ValueError as e:
        print(f"Simulation error: {e}")

    return state_log, control_log

# initial_state = np.array([1., 0., 0., 0., 0., 0., 1.5, 0., 0., 0., 0., 0., 0.])
initial_state = np.hstack(
    (np.array([0., 0., 0.8]),
     .5* np.random.randn(3),
    np.array([0., 0., 0., 0., 0., 0.])
    )
)

ref_state = np.array([0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.])
gravity = mbp.gravity_field().gravity_vector()[2]
ref_action = np.array([-controller.mass * gravity / 4 for i in range(4)] )

0.775


In [163]:


#################
@timeit
def Rollout(x0: np.ndarray, U: np.ndarray, dt: float, timeit: bool = False):
    """
    Simulates the trajectory of states (rollout) for given initial state and control sequence.

    :param x0: initial state, ndarray of shape (nx,)
    :param U: sequence of actions, ndarray of shape (num_time_steps-1, nu)
    :param dt: 

    :return x: sequence of states from rollout, ndarray of shape (num_time_steps, nx)
    """

    X = [x0.copy().tolist()]


    simulator = Simulator(quadrotor)
    simulator.Initialize()
    simulator_context = simulator.get_mutable_context()
    simulator_context.SetContinuousState(x0)

    for u in U:
        # Set input and simulate a small time step

        quadrotor.get_input_port().FixValue(simulator_context, u)
        
        # Use the plant's dynamics (or a numerical integrator) to step forward in time
        # This should ideally be a single step of the plant's integrator
        # simulator = Simulator(self.plant, context)
        sim_time = simulator_context.get_time()

        simulator.AdvanceTo(sim_time + dt)
        x_next = simulator_context.get_continuous_state_vector().CopyToVector()

        X.append(x_next)

        # print(f"time: {sim_time + dt} \nx_next: {x_next} \nu: {u} \n")

    return np.array(X)
#########

# Quadrotor params
# Kinda lazy to hardcode but what can you do ¯\_(ツ)_/¯
L = 0.15  # Length of the arms (m).
kF = 1.0  # Force input constant.
kM = 0.0245  # Moment input constant.
m = 0.775
MoI = np.array([[0.0015, 0, 0,],
                        [0, 0.0025, 0],  
                        [0, 0, 0.0035]])
inv_MoI = np.linalg.inv(MoI)

def ContinuousDynamics(x, u):
        '''
        :param x: state as [x, y, z, yaw, pitch, roll, x_dot, y_dot, z_dot, yaw_rate, pitch_rate, roll_rate]^T
        :param u: action as motor currents
        :return x_dot: state time derivative
        '''
        #State
        euler_angles = x[3:6]
        linear_velocity = x[6:9]
        angular_velocity = x[9:]
        rpy_angular_velocity = np.flip(angular_velocity)
        y,  p,  r = euler_angles.tolist()
        cr, cp, cy = np.cos(r), np.cos(p), np.cos(y)
        sr, sp, sy = np.sin(r), np.sin(p), np.sin(y)

        # Rotation matrix from body frame to world frame
        R_NB = np.array([[      cp*cy,        cy*sp*sr - sy*cr,    cy*sp*cr + sy*sr],
                        [       sy*cp,        sy*sp*sr + cy*cr,    sy*sp*cr - cy*sr],
                        [        -sp,               cp*sr,               cr*cp    ]])

        # =====================Linear Accel Calc===================== #

        # Compute thrust due to rotors in body frame
        uF_Bz = kF * u 
        Faero_B = np.array([0., 0., np.sum(uF_Bz)], dtype = x.dtype) 

        # Compute gravity and world frame
        Fgrav_N = np.array([0, 0, -m * 9.81], dtype = x.dtype)

        # Total forces
        Ftot_N = Fgrav_N + R_NB @ Faero_B

        linear_accel = Ftot_N / m

        # ====================Angular Accel Calc===================== #

        # Compute moments due to thrust 
        Mx_B = L * (uF_Bz[1] - uF_Bz[3])
        My_B = L * (uF_Bz[2] - uF_Bz[0])

        # Compute reaction moments due to actuating motors (Motor direction accounted for in summing)
        uTau_Bz = kM * u
        Mz_B = uTau_Bz[0] - uTau_Bz[1] + uTau_Bz[2] - uTau_Bz[3]

        # Total torque
        Tau_B = np.array([Mx_B, My_B, Mz_B], dtype = x.dtype)

        # Base angular velocity in world, expressed in base coordinates
        M = np.array([[1.,    0.,   -sp],
                        [0.,    cr,  sr*cp],
                        [0.,   -sr,  cr*cp]], dtype = x.dtype)

        w_BN_B = M @ rpy_angular_velocity

        # Compute angular acceleration in body frame, expressed in world coordinates
        alpha_NB_N = R_NB @ inv_MoI @ (Tau_B - np.cross(w_BN_B, MoI @ w_BN_B) )
        
        
        # Compute angular accleration in world coordinates
        Minv = np.array([[cy/cp,      sy/cp,     0.0],
                        [  -sy,         cy,      0.0],
                        [cy/cp*sp,   sy/cp*sp,   1.0]], dtype = x.dtype)
        
        yaw_rate, pitch_rate, roll_rate = angular_velocity

        Mdt =  np.array([[-cy*sp*pitch_rate - sy*cp*yaw_rate,   -cy*yaw_rate,    0.0,],
                         [-sy*sp*pitch_rate + cy*cp*yaw_rate,   -sy*yaw_rate,    0.0],
                         [           -cp*pitch_rate,                 0.0,        0.0]], dtype = x.dtype)
        
        angular_accel = np.flip(Minv @ (alpha_NB_N - Mdt @ rpy_angular_velocity))
        

        return np.hstack((linear_velocity, angular_velocity, linear_accel, angular_accel))     



from maths.forward_integration import RK4
def discrete_dynamics(x0, u, dt):
    xnext = ContinuousDynamics(x0, u)*dt + x0
    # xdot = ContinuousDynamics(x0, u)
    # xnext= RK4(ContinuousDynamics, x0, u, dt )
    roll, pitch, yaw = xnext[3:6]

    # if roll > np.pi:
    #     roll -= 2*np.pi
    # elif roll < -np.pi:
    #     roll += 2*np.pi 

    # if pitch > np.pi/2:
    #     pass
    # elif pitch < -np.pi/2:
    #     pass

    # if yaw > np.pi:
    #     yaw -= 2*np.pi
    # elif yaw < -np.pi:
    #     yaw += 2*np.pi
    return xnext

@timeit
def AnalyticRollout(x0: np.ndarray, U: np.ndarray, dt: float, timeit: bool = False):
    xtraj = x0.copy()
    x = x0.copy()
    for u in U:
        x = discrete_dynamics(x, u, dt)
        xtraj = np.vstack((xtraj, x))
    return xtraj 

from pydrake.examples import QuadrotorPlant


from pydrake.all import ContinuousState, BasicVector

cpp_quadrotor = QuadrotorPlant()
P = np.array(
    [
        [1, 0, 0, 0, 0, 0],
        [0, 1, 0, 0, 0, 0],
        [0, 0, 1, 0, 0, 0],
        [0, 0, 0, 0, 0, 1],
        [0, 0, 0, 0, 1, 0],
        [0, 0, 0, 1, 0, 0],
    ]
)
PP = np.block([[P, np.zeros((6, 6))], [np.zeros((6, 6)), P]])

def CompareDerivatives(x, u):

    # print(x)
    analyticDerivative = ContinuousDynamics(x, u)
    # context = quadrotor_system.CreateDefaultContext()
    # mbpDerivative = context.get_mutable_continuous_state()
    # context.SetContinuousState(x)
    # quadrotor.get_input_port().FixValue(context, u)
    # mbpDerivative = quadrotor.EvalTimeDerivatives(context)

    context = cpp_quadrotor.CreateDefaultContext()
    cpp_quadrotor.get_input_port().FixValue(context, u)
    context.SetContinuousState(PP@x)
    mbpDerivative = PP@cpp_quadrotor.EvalTimeDerivatives(context).CopyToVector()

    return np.linalg.norm(analyticDerivative - mbpDerivative), analyticDerivative, mbpDerivative



In [165]:
np.set_printoptions(linewidth=200)

# print(mbp.GetStateNames())
dt = 0.01
state_log, control_log = simulate(dt, diagram_context, initial_state, ref_state, ref_action, duration = 1.0)
x0 = state_log[0,:]
# x_simulated = Rollout(x0, control_log, dt)
# x_analytic = AnalyticRollout(x0, control_log, dt)

# for x_sim, x_ana in zip(x_simulated, x_analytic):
#     print(f"x_sim: {x_sim}")
#     print(f"x_ana: {x_ana}")
#     print()

for i in range(len(x_analytic)-1):
    err, analDer, mbpDer = CompareDerivatives(state_log[i,:], control_log[i,:])
    # print(np.linalg.norm(analDer[9:]))
    # print(np.linalg.norm(mbpDer[9:]))

    perErr = np.abs(analDer-mbpDer)/mbpDer
    # print(err)
    # print(np.abs(analDer - mbpDer))
    print(np.abs(analDer-mbpDer)/mbpDer)
    # print(np.mean(perErr[-3:]))
    # print(analDer)
    # print(mbpDer)
    print()

[nan nan nan nan nan nan  0.  0.  0.  0. -0.  0.]

[nan nan nan nan nan nan  0.  0.  0.  0. -0.  0.]

[ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00 -0.00000000e+00  0.00000000e+00  2.97226816e-16  3.83257975e-16  2.09356646e-15  2.11278841e-16 -0.00000000e+00  0.00000000e+00]

[ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00 -0.00000000e+00  0.00000000e+00  3.16361387e-16  2.06491795e-16  2.09590616e-15  1.24249429e-16 -0.00000000e+00 -0.00000000e+00]

[ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00 -0.00000000e+00  0.00000000e+00  1.70964449e-16  1.11587614e-16  2.09347115e-15  5.63982958e-16  3.73311693e-16 -0.00000000e+00]

[ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00 -0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00  4.49415127e-16  0.00000000e+00 -2.99622933e-16]

[ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00 -0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00